In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

print(tf.__version__)

2.9.2


In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15418911151662619506
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8697605625131206390
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [5]:
!cat /proc/meminfo

MemTotal:       13297220 kB
MemFree:         8428840 kB
MemAvailable:   11831116 kB
Buffers:          269764 kB
Cached:          3087436 kB
SwapCached:            0 kB
Active:           500704 kB
Inactive:        4015492 kB
Active(anon):        900 kB
Inactive(anon):   926128 kB
Active(file):     499804 kB
Inactive(file):  3089364 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             14400 kB
Writeback:             0 kB
AnonPages:       1159056 kB
Mapped:           660700 kB
Shmem:             11436 kB
KReclaimable:     109372 kB
Slab:             149384 kB
SReclaimable:     109372 kB
SUnreclaim:        40012 kB
KernelStack:        4592 kB
PageTables:        12264 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648608 kB
Committed_AS:    3532856 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       53072 kB
VmallocChunk:          0 kB
Percpu:          

In [6]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs mmio_stale_data retbleed
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power

In [7]:
#loading dataset
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [8]:
import numpy as np

x_train = np.float32(x_train/255.0)
x_test = np.float32(x_test/255.0)
#print(x_train_norm)
#print(x_test_norm)

In [9]:
x_test.shape

(10000, 32, 32, 3)

In [10]:
x_train.shape

(50000, 32, 32, 3)

In [11]:
y_test.shape

(10000, 1)

In [12]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [13]:
#define network
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, activation='relu',kernel_initializer='he_uniform', padding = 'same', input_shape = (32, 32, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size= (2,2), strides=2),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, kernel_size=3, padding = 'same', activation='relu',kernel_initializer='he_uniform'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size = (2,2), strides= 2),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(64, kernel_size= 3, padding = 'same', activation = 'relu',kernel_initializer='he_uniform'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size = (2,2), strides= 2),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation = 'relu', kernel_initializer='he_uniform'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(10, activation = 'softmax')
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 64)       2

In [22]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping  
from keras.models import load_model

In [23]:
es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=50)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [24]:
y_train.shape

(50000, 1)

In [25]:
#train the network with 20% validation
#history = model.fit(x_train, y_train, batch_size = 128, epochs=20, validation_split=0.2)
history = model.fit(x_train, y_train, validation_split=0.2, epochs=30,batch_size=200, verbose=0,callbacks=[es, mc])
saved_model = load_model('best_model.h5')


Epoch 1: val_accuracy improved from -inf to 0.29340, saving model to best_model.h5

Epoch 2: val_accuracy improved from 0.29340 to 0.49590, saving model to best_model.h5

Epoch 3: val_accuracy improved from 0.49590 to 0.62720, saving model to best_model.h5

Epoch 4: val_accuracy improved from 0.62720 to 0.66160, saving model to best_model.h5

Epoch 5: val_accuracy improved from 0.66160 to 0.68900, saving model to best_model.h5

Epoch 6: val_accuracy improved from 0.68900 to 0.70560, saving model to best_model.h5

Epoch 7: val_accuracy improved from 0.70560 to 0.71470, saving model to best_model.h5

Epoch 8: val_accuracy did not improve from 0.71470

Epoch 9: val_accuracy improved from 0.71470 to 0.74690, saving model to best_model.h5

Epoch 10: val_accuracy did not improve from 0.74690

Epoch 11: val_accuracy improved from 0.74690 to 0.75840, saving model to best_model.h5

Epoch 12: val_accuracy improved from 0.75840 to 0.76520, saving model to best_model.h5

Epoch 13: val_accuracy im

In [26]:
#test accuracy and loss
test_loss, test_acc = saved_model.evaluate(x_test, y_test, verbose=2)
print(test_loss, test_acc)

313/313 - 1s - loss: 0.7209 - accuracy: 0.7828 - 1s/epoch - 4ms/step
0.7209190726280212 0.782800018787384


In [27]:
#test accuracy and loss
_, train_acc = saved_model.evaluate(x_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(x_test, y_test, verbose=0)
print('Train: %.4f, Test: %.4f' % (train_acc, test_acc))

Train: 0.9438, Test: 0.7828


In [28]:
predictions = saved_model.predict(x_test)
predictions[1]

print(np.argmax(predictions[1]))
print(y_test[1])
#recall cifar labels: airplane:0, automobile:1, bird:2, cat:3, deer:4, dog:5, frog:6, horse:7, ship:8, truck:9

313/313 [==============================] - 1s 2ms/step
8
[8]


In [29]:
#model.summary()
#import torchvision.models as models
from tensorflow.keras import layers

base_model = tf.keras.applications.DenseNet121(input_shape=(32,32,3),
                                               include_top=False,
                                               weights='imagenet')

29084464/29084464 [==============================] - 0s 0us/step


In [30]:
base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 16, 16, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1/conv[0][0]']             
                                                                                        

In [31]:
last = base_model.get_layer('conv5_block16_concat').output
x = layers.BatchNormalization()(last)
x = layers.Activation('relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(40, activation='relu')(x)
#x = tf.keras.layers.Dense(30, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(base_model.input, outputs)
#for layer in base_model.layers:
 #   layer.trainable = True

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_train, y_train, validation_split=0.2, epochs=50,batch_size=200, verbose=0,callbacks=[es, mc])
saved_model = load_model('best_model.h5')


Epoch 1: val_accuracy improved from -inf to 0.61770, saving model to best_model.h5

Epoch 2: val_accuracy improved from 0.61770 to 0.74960, saving model to best_model.h5

Epoch 3: val_accuracy did not improve from 0.74960

Epoch 4: val_accuracy improved from 0.74960 to 0.80020, saving model to best_model.h5

Epoch 5: val_accuracy did not improve from 0.80020

Epoch 6: val_accuracy improved from 0.80020 to 0.80540, saving model to best_model.h5

Epoch 7: val_accuracy did not improve from 0.80540

Epoch 8: val_accuracy did not improve from 0.80540

Epoch 9: val_accuracy improved from 0.80540 to 0.81650, saving model to best_model.h5

Epoch 10: val_accuracy did not improve from 0.81650

Epoch 11: val_accuracy did not improve from 0.81650

Epoch 12: val_accuracy did not improve from 0.81650

Epoch 13: val_accuracy improved from 0.81650 to 0.82360, saving model to best_model.h5

Epoch 14: val_accuracy did not improve from 0.82360

Epoch 15: val_accuracy did not improve from 0.82360

Epoch 

In [34]:
last = base_model.get_layer('conv5_block16_concat').output
x = layers.BatchNormalization()(last)
x = layers.Activation('relu')(x)
x=Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation = 'relu')(x)
x=Conv2D(512, (3, 3), strides=(1, 1), padding='same',activation = 'relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(40, activation='relu')(x)
#x = tf.keras.layers.Dense(30, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(base_model.input, outputs)
#for layer in base_model.layers:
 #   layer.trainable = True

In [35]:
es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_train, y_train, validation_split=0.2, epochs=50,batch_size=200, verbose=0,callbacks=[es, mc])
saved_model = load_model('best_model.h5')


Epoch 1: val_accuracy improved from -inf to 0.54050, saving model to best_model.h5

Epoch 2: val_accuracy improved from 0.54050 to 0.71760, saving model to best_model.h5

Epoch 3: val_accuracy improved from 0.71760 to 0.76350, saving model to best_model.h5

Epoch 4: val_accuracy improved from 0.76350 to 0.81270, saving model to best_model.h5

Epoch 5: val_accuracy did not improve from 0.81270

Epoch 6: val_accuracy did not improve from 0.81270

Epoch 7: val_accuracy did not improve from 0.81270

Epoch 8: val_accuracy did not improve from 0.81270

Epoch 9: val_accuracy did not improve from 0.81270

Epoch 10: val_accuracy did not improve from 0.81270

Epoch 11: val_accuracy did not improve from 0.81270

Epoch 12: val_accuracy did not improve from 0.81270

Epoch 13: val_accuracy did not improve from 0.81270

Epoch 14: val_accuracy did not improve from 0.81270

Epoch 15: val_accuracy improved from 0.81270 to 0.82670, saving model to best_model.h5

Epoch 16: val_accuracy did not improve fr

In [36]:
sgd = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.8, nesterov=False)

model.compile(optimizer=sgd, loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_train, y_train, validation_split=0.2, epochs=50,batch_size=200, verbose=0,callbacks=[es, mc])
saved_model = load_model('best_model.h5')


Epoch 1: val_accuracy did not improve from 0.84860

Epoch 2: val_accuracy improved from 0.84860 to 0.84910, saving model to best_model.h5

Epoch 3: val_accuracy did not improve from 0.84910

Epoch 4: val_accuracy did not improve from 0.84910

Epoch 5: val_accuracy did not improve from 0.84910

Epoch 6: val_accuracy did not improve from 0.84910

Epoch 7: val_accuracy did not improve from 0.84910

Epoch 8: val_accuracy improved from 0.84910 to 0.85530, saving model to best_model.h5

Epoch 9: val_accuracy improved from 0.85530 to 0.85680, saving model to best_model.h5

Epoch 10: val_accuracy did not improve from 0.85680

Epoch 11: val_accuracy did not improve from 0.85680

Epoch 12: val_accuracy did not improve from 0.85680

Epoch 13: val_accuracy did not improve from 0.85680

Epoch 14: val_accuracy did not improve from 0.85680

Epoch 15: val_accuracy did not improve from 0.85680

Epoch 16: val_accuracy improved from 0.85680 to 0.85820, saving model to best_model.h5

Epoch 17: val_accura